This notebook is for testing the application. Read the comments. You can play around with it. 

Make sure to have your api key and your model name configured in `services/config/personal_constants.py`

In [ ]:
# importing the class from a different file
from services.querier.querier import SqlGenerator
import os

# instantiating the class
test_querier = SqlGenerator(sample_db_loc=os.path.join('data','PostNL_SQLite.sqlite'))

# do some configurations
test_querier.remove_all_training_data() # if there was any, now there is not
test_querier.train_model(train_on_documentation=True, 
                         train_on_ddl=True, 
                         train_on_sql=False, 
                         train_on_question_sql_pairs=True)

In [ ]:
# showing the training data that is in the model
test_querier.training_data

In [ ]:
# some code to play around with
question = """
Hoeveel pakketten heeft elke klant in totaal ooit ontvangen?
"""

sql_code = test_querier.generate_sql(question)

In [ ]:
print(sql_code)

In [ ]:
sample_data = test_querier.generate_sample_data(sql_query=sql_code)
sample_data

In [ ]:
# test loop which compares the data from the generated sql and the correct sql by their shape
# all tests are correct
import json
import pandas as pd

question_sql_file = os.path.join(test_querier.training_data_path, 'question-sql-pairs', 'question-sql-pairs.json')
with open(question_sql_file, 'r') as jf:
    json_file = json.load(jf)

for question, sql in json_file.items():
    try:
        print('Question {}'.format(question), end='\n\n')
        print('Correct SQL: {}'.format(sql), end='\n\n')
        generated_sql = test_querier.generate_sql(question)
        print('Generated SQL: {}'.format(generated_sql), end='\n\n')
        result_correct_sql = test_querier.generate_sample_data(generated_sql)
        result_generated_sql = test_querier.generate_sample_data(sql)
        print('Result of test: {}'.format(result_correct_sql.shape==result_generated_sql.shape), end='\n\n')
    except pd.errors.DatabaseError:
        print('One of the SQL queries did not work')
    print('\n\n\n\n')
    